In [4]:
import pandas as pd

data = pd.read_pickle('/home/p1dg/generation_bot/data/datasets/prepaired_data.df')
data = data[~(data['companions'].apply(len) == 0)].reset_index(drop=True)

data.sample(10)

,Season,Episode,speaker,query,companions,close_reply,neutral_reply,bad_reply,context
4861,Season 2,Episode 7,daenerys targaryen,"We can't find her. She must be dead, too. I le...","[varys, ros, varys, ros, varys]","[And, unlike your current employer, I protect ...",[The Queen in the North! The Queen in the Nort...,"[Shh. I'm right here with you., A small gift f...",You wouldn't remember me if we had?. I meet a ...
15695,Season 6,Episode 7,sparrow,Congress does not require desire on the woman'...,"[sandor, beric, sandor, arya stark, beric]","[Oh, for fuck's sake. May as well be at a bloo...",[Forgive me. Sometimes the truth path is hard ...,"[She doesn't like your queen, does she?, And I...","When she woke up, you know what she did? Suckl..."
15050,Season 6,Episode 4,missandei,How many days were you a slave?,"[jorah mormont, jon snow, jorah mormont, jon s...","[and JON glance at each other., I'm glad he di...","[Not long enough to understand, Long enough to...","[Aye., They are. In a tyrant's grip. Whose fau...",He was a good man. He deserved a better son. W...
10966,Season 4,Episode 7,bronn,My lonesome bachelor days are over. I'm to wed...,"[daenerys targaryen, barristan, daenerys targa...","[He sang to them?, Yes. Rhaegar would take a s...",[Lollys Stokeworth? She doesn't strike me as y...,"[You've ordered us to be night watchmen., You ...",What business would you have with the Queen?. ...
1240,Season 1,Episode 4,bronn,You can have my room.,"[maester luwin, bran stark, maester luwin, bra...","[I still can't tell you, but she will be home ...","[There's a clever man. You can manage food, I ...",[You will do no such thing. I will see each an...,Your mother had to leave Winterfell to protect...
8964,Season 3,Episode 10,melisandre,I take no credit. I have faith and my faith ha...,"[jojen, bran stark, jojen, meera, jojen]","[You mustn't let anything stop you., They alre...","[Your Grace, the world has got so far bent. I'...",[I don't want you to be a hero. Heroes do stup...,Bloody ridiculous. Help me with these straps.....
7911,Season 3,Episode 7,margaery tyrell,"It can be. We're very complicated, you know. P...","[jaime lannister, joffrey lannister, jaime lan...","[Margaery Tyrell, I've heard., By my leave. Th...",[How do you know all this? Did your mother tea...,"[I'm the weakness?, I owe you my life, Ser., I...","All the Kingsguard will be on duty, of course...."
7403,Season 3,Episode 5,shireen,At the beginning. This word is Aegon. When you...,"[bran stark, meera, bran stark, walder frey, r...",[Just a cook in the Night's Watch. He was angr...,"[A rather short siege., It was a bitch of a si...",[I was wrong to support Renly's claim. I know ...,Maybe we shouldn't stay.. You'd rather be out ...
9184,Season 4,Episode 1,shae,I know. I know. We have come to a dangerous pl...,"[tywin lannister, jaime lannister, tywin lanni...",[When the testimony's concluded and a guilty v...,"[Of course I haven't forgotten., I haven't for...",[Joffrey has made this poor girl's life misera...,"No, he'll be punished accordingly.. Once you s..."
13994,Season 5,Episode 8,sam,You're a good lad.,"[jon snow, sansa stark, jon snow, sansa stark,...","[No. No, he never cursed in front of his girls...",[Wanted to ask you something. If you've got th...,"[Rise, Ser Barristan., I came to free you. If ...",At what?. At ruling.. No.. Why are you laughin...


In [5]:

import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
import warnings
import random

/home/p1dg/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
trained_model_dir = '/home/p1dg/generation_bot/models_storage/checkpoint-700'

config = PeftConfig.from_pretrained(trained_model_dir)

In [7]:
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


trained_model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	quantization_config=bnb_config,
	trust_remote_code=True,
	device_map='auto'

	)

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_answer(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return text_output.split("[/INST]")[1].split("\n\n")[0]

def get_test_prompt(df):
    use_col = ['speaker', 'query', 'context', "companions", "close_reply"]
    j = random.randint(0, df.shape[0])
    id = random.randint(0, len(df[use_col].iloc[j]["companions"]))
    query = df[use_col].iloc[j]['query']
    role = df[use_col].iloc[j]['companions'][id]
    context = df[use_col].iloc[j]['context']
    reply = df.iloc[j][use_col]["close_reply"][id]
    prompt = f"[INST]"
    prompt += f'Use the given context to guide your an about the query like indicated in your role'
    prompt += f"query: {query}\n\n"
    prompt += f"context: {context}\n\n"
    prompt += f"your role: {role}\n\n"
    prompt += f'answer:[/INST]'

    return query, role, reply, prompt

In [13]:
query, role, reply, prompt = get_test_prompt(data)
answer = generate_answer(prompt, trained_model)

print(f"\n\nQuery: {query}\n")
print(f"Trained_model reply with role: {role}\n")

print(answer)
print()
print(f'Answer in df: {reply}')



Query: Who then? You?

Trained_model reply with role: brienne

You have been chosen by the king to serve him as his personal bodyguard. Your task is simple: protect the crown from any attackers who dare cross its borders. But there will also come times when it may not be safe enough to do so. Sooner or later, someone must die. It can only happen if you fail to act quickly. In this case, you should take up residence within the walls of the city itself. There, you will find yourself surrounded by guards armed with swords and shields. They will keep watch over your domain, but they won't stop until you surrender. As soon as that happens, you will begin to feel the weight of responsibility. This is no time to relax. A great battle has just begun!

Answer in df: I spent three days looking for her She disappeared


In [14]:
query, role, reply, prompt = get_test_prompt(data)
answer = generate_answer(prompt, trained_model)

print(f"\n\nQuery: {query}\n")
print(f"Trained_model reply with role: {role}\n")

print(answer)
print()
print(f'Answer in df: {reply}')



Query: You're right. It's not my wedding. Anyway, from what I can tell, the dressmakers in Highgarden will be far superior to the ones in King's Landing. They'd never make me anything as dull as this for my wedding. Loras likes green and gold brocade.

Trained_model reply with role: tyrion lannister

I do. But she doesn't really care that much. And if she did, it would have been different.

Answer in df: Shh. I don't know what you're talking about.
